In [1]:
import pdfquery
import datetime
import pandas as pd

In [2]:
def extract_text(l):
    return "".join([k for k in l if not k.isdigit() and not k in [".", ",", "+"]]).rstrip()
def extract_numbers(l, el=None):
#     lis = "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ")
#     print(lis)
#     print([m for m in lis if len(m) > 0])
    result = [int(m) for m in "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ") if len(m) > 0]
    if len(result) < 4:
        result += [0]*(4-len(result))
    if el is None:
        return result
    else:
        return result[el]
def count_numbers(l):
    c = 0
    try:
        c += sum([1 for k in l if k.isdigit()])
        return c
    except:
        return c
def extract_number(l):
    result = "".join([k for k in l.values[0] if k.isdigit()]).lstrip().rstrip()
    if len(result) > 0:
        return int(result)
    else:
        return 0
# df.loc[:, ['confirmed']].apply(extract_number, axis=1)

In [3]:
def load_pdf(date, path="data/RKI-reports/{0}-{1}.pdf", lang="en"):
    from tabula import read_pdf
    path = path.format(date, lang)
    print(path)
    df = read_pdf(path, pages=[2])
    df = df[0].iloc[4:]
    try:
        print("Extracting single data column")
        df.columns = ['data']
        df['c'] = df.loc[:, 'data'].apply(count_numbers)
        df = df[df.c != 0]
    
        df['land'] = df.loc[:, 'data'].apply(extract_text)
        df['confirmed'] = df.loc[:, 'data'].apply(extract_numbers, args=[0])
        df['daily'] = df.loc[:, 'data'].apply(extract_numbers, args=[1])
        df['per_mil'] = df.loc[:, 'data'].apply(extract_numbers, args=[2])
        df['dead'] = df.loc[:, 'data'].apply(extract_numbers, args=[3])
    except:
        if len(df.columns) == 4:
            print("Extracting 4 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'dead']
            df['per_mil'] = 0
            df = df.loc[:, ['land', 'confirmed', 'daily', 'per_mil', 'dead']]
        elif len(df.columns) == 5:
            print("Extracting 5 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'per_mil', 'dead']
        elif len(df.columns) == 6:
            print("Extracting 6 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'per_mil', 'dead', 'dead_per_100k']
        else:
            print(f"Falied to exctract {len(df.columns)} data columns")
            print(df)
            raise Exception
        df.drop(['dead_per_100k'], axis=1, inplace=True)
        df['c'] = df.loc[:, 'confirmed'].apply(count_numbers)
        df = df.loc[df.c != 0, :]
        df.loc[:,['confirmed']] = df.loc[:,['confirmed']].apply(extract_number, axis=1)
        df.loc[:,['daily']] = df.loc[:,['daily']].apply(extract_number, axis=1)
        df.loc[:,['dead']] = df.loc[:,['dead']].apply(extract_number, axis=1)
    df.loc[df.land.isnull() == True, 'land'] = 'Mecklenburg-Western Pomerania'
    df = df.loc[(df.land.str.contains('cases') == False) & (df.land != 'Total') 
                & (df.land.str.contains('Gesamt') == False), :]
    try:
        df['date'] = datetime.datetime.strptime(date, "%Y-%m-%d")
    except:
        df['date'] = date
    return df

# Test Load

In [768]:
df = load_pdf('2020-04-28', lang="de").head()
type(df)

data/RKI-reports/2020-04-28-de.pdf


Got stderr: Apr 28, 2020 1:44:28 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
Apr 28, 2020 1:44:28 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
Apr 28, 2020 1:44:29 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored



Extracting single data column
Extracting 6 data columns


/Users/chekanskiy/Documents/projects/covid-19/venv/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


pandas.core.frame.DataFrame

# Batch Load

In [519]:
df = pd.DataFrame(columns=['data', 'land', 'confirmed', 'daily', 'per_mil', 'dead'])
list_df = []
start='2020-03-18'
end='2020-04-17'
for d in pd.date_range(start, end).date:
    list_df.append(load_pdf(d, path="data/RKI-reports/{0}-{1}.pdf", lang="en"))

data/RKI-reports/2020-03-18-en.pdf
data/RKI-reports/2020-03-19-en.pdf
data/RKI-reports/2020-03-20-en.pdf
data/RKI-reports/2020-03-21-en.pdf
data/RKI-reports/2020-03-22-en.pdf
data/RKI-reports/2020-03-23-en.pdf
data/RKI-reports/2020-03-24-en.pdf
data/RKI-reports/2020-03-25-en.pdf
data/RKI-reports/2020-03-26-en.pdf
data/RKI-reports/2020-03-27-en.pdf
data/RKI-reports/2020-03-28-en.pdf
data/RKI-reports/2020-03-29-en.pdf
data/RKI-reports/2020-03-30-en.pdf
data/RKI-reports/2020-03-31-en.pdf
data/RKI-reports/2020-04-01-en.pdf
data/RKI-reports/2020-04-02-en.pdf
data/RKI-reports/2020-04-03-en.pdf
data/RKI-reports/2020-04-04-en.pdf
data/RKI-reports/2020-04-05-en.pdf
data/RKI-reports/2020-04-06-en.pdf
data/RKI-reports/2020-04-07-en.pdf
data/RKI-reports/2020-04-08-en.pdf
data/RKI-reports/2020-04-09-en.pdf
data/RKI-reports/2020-04-10-en.pdf
data/RKI-reports/2020-04-11-en.pdf
data/RKI-reports/2020-04-12-en.pdf
data/RKI-reports/2020-04-13-en.pdf
data/RKI-reports/2020-04-14-en.pdf
data/RKI-reports/202

## Batch Save

In [544]:
result_df = pd.concat(list_df).drop(['daily', 'c', 'per_mil'], axis=1)

In [545]:
result_df.to_csv('data/RKI-reports.csv', index=False)

-----

-----

# Updating Reports

In [39]:
df_new = load_pdf('2020-05-02', lang="en")
df_new.drop(['c', 'daily', 'per_mil'], axis=1, inplace=True)

data/RKI-reports/2020-05-02-en.pdf
Extracting single data column
Extracting 6 data columns


/Users/chekanskiy/Documents/projects/covid-19/venv/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [40]:
result_all = pd.read_csv('data/RKI-reports.csv')
result_all['date'] = result_all['date'].astype('datetime64[ns]')

In [41]:
result_concat = pd.concat([result_all, df_new]).sort_values('date', ascending=False)
result_concat.head()

,land,confirmed,dead,date,data
21,Thuringia,2335,93,2020-05-02,NaN
12,Mecklenburg-Western Pomerania,695,18,2020-05-02,NaN
5,Bavaria,42658,1885,2020-05-02,NaN
6,Berlin,5943,152,2020-05-02,NaN
7,Brandenburg,2886,122,2020-05-02,NaN


In [42]:
result_concat = result_concat[result_concat.land != 'Federal State Total Number Number of Cases/ Number of'].drop_duplicates()

In [43]:
result_concat.loc[result_concat['land'].str.contains('Schleswig-Holstein') == True, 'land'] = 'Schleswig-Holstein'
result_concat.loc[result_concat['land'].str.contains('Baden-W') == True, 'land'] = 'Baden-Wuerttemberg'
result_concat.loc[result_concat['land'].str.contains('Saxony-A') == True, 'land'] = 'Saxony-Anhalt'
result_concat.loc[result_concat['land'].str.contains('Sachsen-Anhalt') == True, 'land'] = 'Saxony-Anhalt'
result_concat.loc[result_concat['land'].str.contains('Mecklenburg-W') == True, 'land'] = 'Mecklenburg-Western Pomerania'

result_concat.loc[result_concat['land'].str.contains('Nordrhein-Westfalen') == True, 'land'] = 'North Rhine-Westphalia'
result_concat.loc[result_concat['land'].str.contains('Sachsen') == True, 'land'] = 'Saxony'
result_concat.loc[result_concat['land'].str.contains('Rheinland-Pfalz') == True, 'land'] = 'Rhineland-Palatinate'
result_concat.loc[result_concat['land'].str.contains('Niedersachsen') == True, 'land'] = 'Lower Saxony'
result_concat.loc[result_concat['land'].str.contains('Hessen') == True, 'land'] = 'Hesse'
result_concat.loc[result_concat['land'].str.contains('Thüringen') == True, 'land'] = 'Thuringia'

In [44]:
result_concat.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50)

,confirmed
land,
Brandenburg,46
Bremen,46
Hamburg,46
Hesse,46
Lower Saxony,46
North Rhine-Westphalia,46
Rhineland-Palatinate,46
Saarland,46
Saxony,46


In [45]:
len(result_concat)

727

In [46]:
result_concat = result_concat.loc[~((result_concat.land == 'Saxony') & (result_concat.date=='2020-03-30') & (result_concat.data.str.contains('Anhalt') == True)), :]

In [47]:
result_concat = result_concat.drop_duplicates()

In [48]:
len(result_concat)

727

In [49]:
result_concat.to_csv('data/RKI-reports.csv', index=False)

---

In [50]:
result_concat.loc[:, ['land', 'date', 'confirmed']].groupby(['land', 'date']).count().sort_values(['confirmed'], ascending=False).head()

confirmed
land                 date                 
Baden-Wuerttemberg   2020-03-19          1
Rhineland-Palatinate 2020-04-14          1
                     2020-04-16          1
                     2020-04-17          1
                     2020-04-18          1

In [51]:
# result_concat.loc[(result_concat.land == 'Mecklenburg-Western Pomerania') & (result_concat.date=='2020-04-27')
# #                   & (result_concat.data.str.contains('Anhalt') == True)
#                   , :]

In [52]:
# fixed_cresult.loc[fixed_cresult['land'].str.contains('Schleswig-Holstein') == True, 'land'] = 'Schleswig-Holstein'
# fixed_cresult.loc[fixed_cresult['land'].str.contains('Baden-W') == True, 'land'] = 'Baden-Wuerttemberg'
# fixed_cresult.loc[fixed_cresult['land'].str.contains('Saxony-A') == True, 'land'] = 'Saxony-Anhalt'
# fixed_cresult.loc[fixed_cresult['land'].str.contains('Mecklenburg-W') == True, 'land'] = 'Mecklenburg-Western Pomerania'

In [53]:
# fixed_cresult.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50)


---

In [732]:
# fixed_cresult.to_csv('data/RKI-reports.csv', index=False)

----